# インバウンド認証

AgentCore Identity を使用すると、AgentCore Runtime でエージェントやツールを呼び出すユーザーおよびアプリケーションのインバウンドアクセス（インバウンド認証）を検証したり、AgentCore Gateway ターゲットへのアクセスを検証したりできます。また、エージェントから外部サービスまたは Gateway ターゲットへのアウトバウンドアクセス（アウトバウンド認証）を保護します。既存のIDプロバイダー（Amazon Cognitoなど）と統合しながら、独立して動作するエージェント、またはユーザーに代わって動作するエージェント（OAuth経由）の権限境界を強制します。

インバウンド認証は、AgentCore Runtime、AgentCore Gateway、またはその他の環境でホストされているエージェントやツールを呼び出そうとする呼び出し元を検証します。インバウンド認証は、IAM（SigV4資格情報）またはOAuth認可で動作します。

デフォルトでは、Amazon Bedrock AgentCore はIAM資格情報を使用し、エージェントへのユーザーリクエストはユーザーのIAM資格情報で認証されます。OAuthを使用する場合は、AgentCore Runtime リソースまたは AgentCore Gateway エンドポイントを設定する際に以下を指定する必要があります：

- OAuthディスカバリーサーバーURL — OpenID Connectディスカバリー URL のパターン ^.+/\.well-known/openid-configuration$ に一致する文字列

- 許可されたオーディエンス — JWTトークンの許可されたオーディエンスのリスト

- 許可されたクライアント — 許可されたクライアント識別子のリスト

AgentCore CLI を使用する場合、**configure** コマンドを使用するときに AgentCore Runtime の認可タイプ（およびOAuthディスカバリーサーバー）を指定できます。CreateAgentRuntime オペレーションと Amazon Bedrock AgentCore コンソールも使用できます。Gateway を作成する場合は、CreateGateway オペレーションまたはコンソールを使用します。

ユーザーがエージェントを使用する前に、クライアントアプリケーションはユーザーに OAuth オーソライザーで認証させる必要があります。クライアントはベアラートークンを受け取り、呼び出しリクエストでエージェントに渡します。受信時にエージェントは、アクセスを許可する前に認可サーバーでトークンを検証します。


## 概要

このチュートリアルでは、01-AgentCore-runtime でデプロイしたエージェントを変更し、Cognito を ID プロバイダーとして使用するインバウンド認証用に設定します。1人のユーザーとアプリクライアントを持つ Cognito ユーザープールをセットアップします。Cognito ユーザープールを使用したインバウンド認証で Amazon Bedrock AgentCore Runtime を使用して既存のエージェントをホストする方法を学びます。

### チュートリアルアーキテクチャ

<div style="text-align:center">
    <img src="images/inbound_auth_cognito.png" width="90%"/>
</div>

### チュートリアル詳細


| 情報                | 詳細                                                                             |
|:--------------------|:--------------------------------------------------------------------------------|
| チュートリアルタイプ | 対話型                                                                          |
| エージェントタイプ   | シングル                                                                        |
| エージェントフレームワーク | Strands Agents                                                            |
| LLMモデル           | Anthropic Claude Haiku 4.5                                                      |
| チュートリアルコンポーネント | AgentCore Runtime でのエージェントホスティング。Strands Agent と Amazon Bedrock Model の使用 |
| チュートリアル分野   | クロスバーティカル                                                              |
| 難易度              | 簡単                                                                            |
| インバウンド認証     | Cognito                                                                         |
| 使用SDK             | Amazon BedrockAgentCore Python SDK および boto3                                  |



### チュートリアルの主な機能

* Amazon Cognito を使用したインバウンド認証で Amazon Bedrock AgentCore Runtime にエージェントをホスティング
* Amazon Bedrock モデルの使用
* Strands Agents の使用

## 前提条件

このチュートリアルを実行するには以下が必要です：
* Python 3.10+
* AWS 資格情報
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker が実行中であること

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## 認証用の Amazon Cognito のセットアップ

アプリクライアントと1人のテストユーザーを持つ Cognito ユーザープールをプロビジョニングします。デプロイされた MCP サーバーにアクセスするための JWT トークンを提供するために Amazon Cognito を使用します。そのために、`utils` スクリプトの `setup_cognito_user_pool` サポート関数を使用します。

注意：Cognito access_token は2時間のみ有効です。access_token の有効期限が切れた場合は、`reauthenticate_user` メソッドを使用して別の access_token を取得できます。

In [ ]:
import sys
import os

# 現在のノートブックのディレクトリを取得
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# sys.path に追加
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from utils import setup_cognito_user_pool, reauthenticate_user

In [ ]:
print("Amazon Cognito ユーザープールをセットアップ中...")
cognito_config = setup_cognito_user_pool()
print("Cognito セットアップ完了 ✓")

## AgentCore Runtime へのデプロイ用にエージェントを準備する

### Amazon Bedrock モデルを使用した Strands Agents
01-AgentCore-runtime チュートリアルで作成した Strands Agent から始めて、Amazon Cognito を ID プロバイダーとして使用するインバウンド認証で設定します。

In [ ]:
%%writefile strands_claude.py
from strands import Agent, tool
from strands_tools import calculator # calculator ツールをインポート
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel

app = BedrockAgentCoreApp()

# カスタムツールを作成
@tool
def weather():
    """ 天気を取得 """ # ダミー実装
    return "sunny"


model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="あなたは親切なアシスタントです。簡単な数学計算と天気を教えることができます。"
)

@app.entrypoint
def strands_agent_bedrock(payload):
    """
    ペイロードでエージェントを呼び出す
    """
    user_input = payload.get("prompt")
    print("ユーザー入力:", user_input)
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

## エージェントを AgentCore Runtime にデプロイする

`CreateAgentRuntime` オペレーションは包括的な設定オプションをサポートしており、コンテナイメージ、環境変数、暗号化設定を指定できます。また、クライアントがエージェントと通信する方法を制御するためのプロトコル設定（HTTP、MCP）と認可メカニズムも設定できます。

**注意：** 運用のベストプラクティスは、コードをコンテナとしてパッケージ化し、CI/CD パイプラインと IaC を使用して ECR にプッシュすることです

このチュートリアルでは、Amazon Bedrock AgentCode Python SDK を使用してアーティファクトを簡単にパッケージ化し、AgentCore Runtime にデプロイします。

### AgentCore Runtime デプロイメントの設定

次に、スターターツールキットを使用して、エントリポイント、作成した実行ロール、requirements ファイルで AgentCore Runtime デプロイメントを設定します。また、起動時に Amazon ECR リポジトリを自動作成するようにスターターキットを設定します。

設定ステップ中に、アプリケーションコードに基づいて Dockerfile が生成されます

**重要** - 前のステップの Cognito ディスカバリー URL と Cognito アプリクライアント ID を更新してください。

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

discovery_url = cognito_config.get("discovery_url")

client_id = cognito_config.get("client_id")

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_claude.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_agent_inbound_identity",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "discoveryUrl": discovery_url,
            "allowedClients": [client_id]
        }
    }
)
response


## AgentCore 設定の確認

In [ ]:
!cat .bedrock_agentcore.yaml

### AgentCore Runtime へのエージェントの起動

Dockerfile ができたので、エージェントを AgentCore Runtime に起動しましょう。これにより、Amazon ECR リポジトリと AgentCore Runtime が作成されます

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()
launch_result

### AgentCore Runtime ステータスの確認
AgentCore Runtime をデプロイしたので、デプロイメントステータスを確認しましょう

In [ ]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### 認可なしで AgentCore Runtime を呼び出す

最後に、ペイロードで AgentCore Runtime を呼び出すことができます。次のセルを実行すると、**"AccessDeniedException: An error occurred (AccessDeniedException) when calling the InvokeAgentRuntime operation: Agent is configured for a different authorization token type"** というエラーが表示されます。

<div style="text-align:left">
    <img src="images/invoke.png" width="75%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "今の天気はどうですか？"})
invoke_response

### 認可ありで AgentCore Runtime を呼び出す

正しい認可トークンタイプでエージェントを呼び出しましょう。この場合、Cognito アクセストークンになります。**「Cognito ユーザープールのプロビジョニング」** セルからアクセストークンをコピーしてください

In [ ]:
bearer_token = reauthenticate_user(cognito_config.get("client_id"))
invoke_response = agentcore_runtime.invoke(
    {"prompt": "今の天気はどうですか？"}, 
    bearer_token=bearer_token
)
invoke_response

## クリーンアップ（オプション）

作成した AgentCore Runtime をクリーンアップしましょう

In [ ]:
from boto3.session import Session
import boto3
boto_session = Session()

agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# おめでとうございます！